In [1]:
!pip install -U altair==5.2.0 vega_datasets==0.9.0



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd 
import altair as alt
from vega_datasets import data

In [3]:
def display_df(df, rows=pd.options.display.max_rows, cols=pd.options.display.max_columns):
    """Displays n rows and cols from df"""
    with pd.option_context("display.max_rows", rows,
                           "display.max_columns", cols):
        display(df)

In [16]:
cities_fp = os.path.join('data', 'climate_cities.csv')
cities = pd.read_csv(cities_fp)
# display_df(cities, rows=cities.shape[0], cols=cities.shape[1])
cities.head()

,city,lat,lon,month,monthnum,sunshine,rain
0,San Diego,32.715736,-117.161087,Jan,1,217,1.53
1,San Diego,32.715736,-117.161087,Feb,2,255,0.15
2,San Diego,32.715736,-117.161087,Mar,3,234,0.57
3,San Diego,32.715736,-117.161087,Apr,4,236,1.01
4,San Diego,32.715736,-117.161087,May,5,277,0.02
5,San Diego,32.715736,-117.161087,Jun,6,293,0.03
6,San Diego,32.715736,-117.161087,Jul,7,253,0.07
7,San Diego,32.715736,-117.161087,Aug,8,261,0.12
8,San Diego,32.715736,-117.161087,Sep,9,242,0.78
9,San Diego,32.715736,-117.161087,Oct,10,262,1.81


In [73]:
import altair as alt
import pandas as pd

# Load data
cities_fp = 'data/climate_cities.csv'
cities = pd.read_csv(cities_fp)

# Sort data by latitude in descending order
cities = cities.sort_values(by='lat', ascending=False)

# Calculate weighted weather quality with two standard deviations
sunshine_std = cities.groupby('city')['sunshine'].transform('std')
rain_std = cities.groupby('city')['rain'].transform('std')

cities['weather_quality_scaled'] = ((cities['sunshine'] - cities['sunshine'].mean()) / sunshine_std -
                                     (cities['rain'] - cities['rain'].mean()) / rain_std)

# Normalize the data for z-score normalization
cities['sunshine_zscore'] = (cities['sunshine'] - cities['sunshine'].mean()) / sunshine_std
cities['rain_zscore'] = (cities['rain'] - cities['rain'].mean()) / rain_std
cities['weather_quality_zscore'] = (cities['weather_quality_scaled'] - cities['weather_quality_scaled'].mean()) / cities['weather_quality_scaled'].std()

# Define base chart with higher resolution and larger scale
base = alt.Chart(cities, width=280, height=300).encode(
    x=alt.X('monthnum:N', axis=alt.Axis(title='Months', labelAngle=0))  # Set labelAngle to 0 for vertical labels
)

# Bar chart for standardized weather quality
bar_chart = base.mark_bar().encode(
    y=alt.Y('weather_quality_zscore:Q', axis=alt.Axis(title='Weather Quality')),
    color=alt.Color('lat:Q', scale=alt.Scale(scheme='blues', domain=[cities['lat'].min(), cities['lat'].max()], range=['yellow', 'purple']), title='Latitude (Cooler to Warmer)')
)

# Line plot for standardized sunshine and rainfall
line_chart = base.mark_line(color='red').encode(
    y=alt.Y('sunshine_zscore:Q', axis=alt.Axis(title='Z-scores of Sunshine'))
) + base.mark_line(color='blue').encode(
    y=alt.Y('rain_zscore:Q', axis=alt.Axis(title='Rainfall'))
)

# Combine the charts and facet by city
final_chart = alt.layer(bar_chart, line_chart).facet(
    facet=alt.Column('city:N'),
    columns=3
).resolve_scale(
    y='independent'
)

# Show the chart
final_chart

alt.FacetChart(...)

In [48]:
import altair as alt
import pandas as pd

# Load data
cities_fp = 'data/climate_cities.csv'
cities = pd.read_csv(cities_fp)

# Calculate weighted weather quality with two standard deviations
sunshine_std = cities.groupby('city')['sunshine'].transform('std')
rain_std = cities.groupby('city')['rain'].transform('std')

cities['weather_quality_scaled'] = ((cities['sunshine'] - cities['sunshine'].mean()) / sunshine_std -
                                     (cities['rain'] - cities['rain'].mean()) / rain_std)

# Normalize the data for z-score normalization
cities['sunshine_zscore'] = (cities['sunshine'] - cities['sunshine'].mean()) / sunshine_std
cities['rain_zscore'] = (cities['rain'] - cities['rain'].mean()) / rain_std
cities['weather_quality_zscore'] = (cities['weather_quality_scaled'] - cities['weather_quality_scaled'].mean()) / cities['weather_quality_scaled'].std()

# Define base chart with higher resolution and larger scale
base = alt.Chart(cities, width=280, height=300).encode(
    x=alt.X('monthnum:N', axis=alt.Axis(title='Months', labelAngle=0))  # Set labelAngle to 0 for vertical labels
)

# Bar chart for standardized weather quality
bar_chart = base.mark_bar(color='gold').encode(
    y=alt.Y('weather_quality_zscore:Q', axis=alt.Axis(title='Weather Quality'))
)

# Line plot for standardized sunshine and rainfall
line_chart = base.mark_line(color='red').encode(
    y=alt.Y('sunshine_zscore:Q', axis=alt.Axis(title='Z-scores of Sunshine'))
) + base.mark_line(color='blue').encode(
    y=alt.Y('rain_zscore:Q', axis=alt.Axis(title='Rainfall'))
)

# Combine the charts and facet by city
final_chart = alt.layer(bar_chart, line_chart).facet(
    facet=alt.Column('city:N'),
    columns=3
).resolve_scale(
    y='independent'
)

# Show the chart
final_chart


alt.FacetChart(...)

In [24]:
cities[['sunshine', 'rain']].max(axis=1)

0     217.0
1     255.0
2     234.0
3     236.0
4     277.0
      ...  
67    263.0
68    216.0
69    215.0
70    212.0
71    209.0
Length: 72, dtype: float64

## What is the weather quality over different cities?
